#Preprocess
This notebook preprocess the data. The basic steps are:

   - Load and join the inputs
   - Split the columns into column groups. That will make it easier to process different sets of columns
   - Preprocess the groups, binarizing, normalizing and creating new features as necessary
   - Save each group into a different file inside **Processed** folder
   - Save the index to the train and test dataset into **Processed** folder
    

In [1]:
import sklearn
import pandas as pd
import mylib.utils as mu
from mylib.utils import (print_time, get_numerical_cols, years_passed, days_passed, 
    save_fields_dict, load_fields_dict, Scaler, binarize)

### Load the input and join databases

In [2]:
# load the datasets
application_raw = pd.read_excel('./input/onsite homework v1.xlsx', 1)
loan_raw = pd.read_excel('./input/onsite homework v1.xlsx', 2)
print_time('Loaded information. Loan #: %d Applications #:%d' %(loan_raw.index.size, application_raw.index.size))

02:51:07 10/08/15 BRT - Loaded information. Loan #: 1285 Applications #:647


In [3]:
# create the customer id on loan to join with application
print_time('Creating customer_id field to join loan and application')
loan = loan_raw.copy()
loan['customer_id']=loan.apply(lambda l: l[0][:-3].lower(), axis = 1)
loan['loan_no']=loan.apply(lambda l: int(l[0][-2:]), axis = 1)

02:51:07 10/08/15 BRT - Creating customer_id field to join loan and application


In [4]:
# found that there is 16 with 2 loans. will keep only the second
idx_more_than_once = (loan.groupby(['customer_id'])['flgGood'].count() >1)
idx_more_than_once = idx_more_than_once[idx_more_than_once == True]
print_time('Number of clients with 2 loans %d' %idx_more_than_once.count())
print_time('Removing first loan. Nr of lines %d' % loan.idLoan.count())
loan.drop(loan[loan.customer_id.isin(idx_more_than_once.index)][loan.loan_no ==1].index, inplace = True)
print_time('Removed first loan. Nr of lines %d' %loan.idLoan.count())

02:51:07 10/08/15 BRT - Number of clients with 2 loans 16
02:51:07 10/08/15 BRT - Removing first loan. Nr of lines 1285
02:51:07 10/08/15 BRT - Removed first loan. Nr of lines 1269


/usr/local/lib/python2.7/site-packages/pandas/core/frame.py:1825: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  "DataFrame index.", UserWarning)


In [5]:
# preprocess application
print_time('Preprocessing application')
application = application_raw.copy()

02:51:07 10/08/15 BRT - Preprocessing application


In [6]:
# create data 
data = pd.merge(application, loan, on='customer_id')
print_time('Final data created. Nr of lines %d' %data.index.size)

02:51:07 10/08/15 BRT - Final data created. Nr of lines 634


In [7]:
# 13 lines lost. Check if it is correct
print_time('Number of lines discarded %d' 
      %application[~application.customer_id.isin(data.customer_id)].customer_id.count())

02:51:07 10/08/15 BRT - Number of lines discarded 13


In [8]:
# target variable
data.rename(columns={'flgGood':'target'}, inplace=True)
data['target'] = [1 if d== 'Good' else 0 for d in data.target]
#del data['target']

### Create group of columns

In [9]:
# cols names helpers
cols=dict(
    raw_scores = [ col for col in data.columns.tolist() if ('raw' in col)],
    personal = [u'Facebook profile duration', u'residence_duration', u'home_phone_type',
                 u'other_phone_type', u'Gender_facebook', u'birth_date', u'Title'],
    location = [u'City', u'State',],
    address = ['CEP', 'StreetAddress', 'Latitude','Longitude'],
    #dates = [u'application_when',  u'birth_date'],
    #ids = ['customer_id', 'idLoan'],
    personal_unique = ['CPF', 'Occupation', 'email', 'TelephoneNumber'],
    name = ['GivenName', 'MiddleInitial', 'Surname'],
    others = ['status'],
    balance = [u'residence_rent_or_own', u'monthly_rent_amount', u'monthly_income_amount', u'bank_account_duration'],
    nu_info = ['loan_no', u'Credit_Line_approved', u'Credit_Line_requested', u'application_when']
)
numeric, non_numeric = get_numerical_cols(data)

In [10]:
# dictionary of transformed data
trans_data = {}
print_time('Feature engineering and cleaning')

# set the index to the id so that all child keep the index
data.set_index('customer_id', inplace=True)

# fill the secondary type with string NI - Not Informed
data.other_phone_type.fillna('NI', inplace=True)
data.bank_account_duration.fillna('NI', inplace=True)
#data.Occupation.fillna('NI', inplace=True)

# binarize categoricals
print_time('Processing personal info')
personal = pd.get_dummies(data[cols['personal']])

# transform date of birth to age
personal['age'] = personal.birth_date.apply(years_passed)
del personal['birth_date']

# create a new feature related to money and age
personal['age_money_ratio_class'] = binarize( data[u'monthly_income_amount']/personal[u'age'])

trans_data['personal'] = personal

02:51:07 10/08/15 BRT - Feature engineering and cleaning
02:51:07 10/08/15 BRT - Processing personal info


In [11]:
# scale the numerical mean 0, sd 1
print_time('Normalizing raw scores')
scl = Scaler(cols['raw_scores'])
raw_scores = scl.fit_transform(data)
trans_data['raw_scores'] = raw_scores[cols['raw_scores']]

02:51:07 10/08/15 BRT - Normalizing raw scores


In [12]:
print_time('Processing balance information')
balance = data[cols['balance']].copy()
balance.residence_rent_or_own = [1 if r else 0 for r in balance.residence_rent_or_own]
balance['rent_income_ratio'] = balance.monthly_rent_amount/balance.monthly_income_amount
balance = pd.get_dummies(balance)
trans_data['balance'] = balance

02:51:07 10/08/15 BRT - Processing balance information


In [13]:
print_time('Processing Nu info')
nu_info = data[cols['nu_info']].copy()
nu_info['credit_approved_ratio'] = nu_info.Credit_Line_approved /nu_info.Credit_Line_requested 
nu_info['days_since_application'] = nu_info.application_when.apply(days_passed)
nu_info['credit_income_ratio'] = nu_info.Credit_Line_approved/balance.monthly_income_amount
nu_info['liquid_income_ratio'] = nu_info.Credit_Line_approved/(balance.monthly_income_amount-balance.monthly_rent_amount)
nu_info['money_to_spare_class'] =  binarize(data[u'monthly_income_amount']-data[u'monthly_rent_amount']-nu_info[u'Credit_Line_approved'])
del nu_info['application_when']
trans_data['nu_info'] = nu_info

02:51:07 10/08/15 BRT - Processing Nu info


In [14]:
print_time('Processing location information')
location = data[cols['location']].copy()
location = pd.get_dummies(location)
trans_data['location'] = location

02:51:07 10/08/15 BRT - Processing location information


In [15]:
print_time('Creating new features - scores classes')
scores = pd.DataFrame(index=data.index)
scores_list = [u'raw_unit4_score', u'raw_lexisnexis_score',
        u'raw_TU_score', u'raw_FICO_money_score']
names_list = []
for col in scores_list:
    names_list.append(col+'_class')
    scores[col+'_class'] = binarize(data[col])
trans_data['scores_class'] = scores
cols['scores_class'] = names_list

02:51:07 10/08/15 BRT - Creating new features - scores classes


### Save the data to the files

In [16]:
# save full processed
print_time('Saving data in Processed folder')
data.to_csv('./Processed/full_data.csv',  encoding='utf-8')
data['target'].to_csv('./Processed/target.csv',  encoding='utf-8', header=True)

# save files for each set of generated colums
for k,v in cols.iteritems():
    # save transformed data if exists, else save the original one
    if k in trans_data:
        # replace ' ' from column names with _
        trans_data[k].columns = [ col.replace(' ', '_').replace('-', '_').replace('+', '') 
                                 for col in trans_data[k].columns.tolist() ]
        trans_data[k].to_csv('./Processed/{}.csv'.format(k), encoding='utf-8')
    else:
        data[v].to_csv('./Processed/{}.csv'.format(k), encoding='utf-8')

02:51:07 10/08/15 BRT - Saving data in Processed folder


In [17]:
# save the dictionary for future use
save_fields_dict(cols)

### Save the common seed

In [18]:
#56909 seed to run the example
mu.save_seed(56909)

02:51:08 10/08/15 BRT - Common seed saved: 56909


56909